In [6]:
import pandas as pd
import sqlite3

# Importation du dataset immobilier de Seattle

In [2]:
Seattle_energy_dataframe = pd.read_csv("2015-building-energy-benchmarking.csv",sep=",")

In [3]:
Seattle_energy_dataframe

,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,TaxParcelIdentificationNumber,Location,CouncilDistrictCode,Neighborhood,YearBuilt,...,GHGEmissionsIntensity(kgCO2e/ft2),DefaultData,Comment,ComplianceStatus,Outlier,2010 Census Tracts,Seattle Police Department Micro Community Policing Plan Areas,City Council Districts,SPD Beats,Zip Codes
0,1,2015,NonResidential,Hotel,MAYFLOWER PARK HOTEL,659000030,"{'latitude': '47.61219025', 'longitude': '-122...",7,DOWNTOWN,1927,...,2.64,No,NaN,Compliant,NaN,NaN,14.0,NaN,31.0,18081
1,2,2015,NonResidential,Hotel,PARAMOUNT HOTEL,659000220,"{'latitude': '47.61310583', 'longitude': '-122...",7,DOWNTOWN,1996,...,2.38,No,NaN,Compliant,NaN,NaN,14.0,NaN,31.0,18081
2,3,2015,NonResidential,Hotel,WESTIN HOTEL,659000475,"{'latitude': '47.61334897', 'longitude': '-122...",7,DOWNTOWN,1969,...,1.92,Yes,NaN,Compliant,NaN,NaN,56.0,NaN,31.0,18081
3,5,2015,NonResidential,Hotel,HOTEL MAX,659000640,"{'latitude': '47.61421585', 'longitude': '-122...",7,DOWNTOWN,1926,...,31.38,No,NaN,Compliant,High Outlier,NaN,56.0,NaN,31.0,18081
4,8,2015,NonResidential,Hotel,WARWICK SEATTLE HOTEL,659000970,"{'latitude': '47.6137544', 'longitude': '-122....",7,DOWNTOWN,1980,...,4.02,No,NaN,Compliant,NaN,NaN,56.0,NaN,31.0,19576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3335,50049,2015,Multifamily LR (1-4),Low-Rise Multifamily,PACIFIC CENTER CONDOMINIUM,6599950000,"{'latitude': '47.59950256', 'longitude': '-122...",2,DOWNTOWN,2000,...,0.07,No,NaN,Compliant,NaN,NaN,26.0,NaN,16.0,18379
3336,50055,2015,Multifamily MR (5-9),Mid-Rise Multifamily,IDENTITY APTS 4123,1142001670,"{'latitude': '47.65752471', 'longitude': '-122...",4,NORTHEAST,2014,...,0.61,No,NaN,Compliant,NaN,NaN,60.0,NaN,38.0,18383
3337,50057,2015,Multifamily HR (10+),High-Rise Multifamily,CIRRUS,660000575,"{'latitude': '47.61649845', 'longitude': '-122...",7,DOWNTOWN,2015,...,1.89,No,NaN,Compliant,NaN,NaN,56.0,NaN,7.0,19576
3338,50058,2015,Multifamily LR (1-4),Low-Rise Multifamily,WEDGEWOOD ESTATES BLDG A,6392001040,"{'latitude': '47.68396954', 'longitude': '-122...",4,NORTHEAST,1981,...,0.04,No,NaN,Compliant,NaN,NaN,55.0,NaN,48.0,18792


On constitue le dataframe Building à partir d'une sélection des colonnes du dataset entier.

In [4]:
building_table = Seattle_energy_dataframe[["OSEBuildingID","PropertyName","PrimaryPropertyType","YearBuilt","NumberofBuildings","NumberofFloors","PropertyGFATotal","ENERGYSTARScore","SiteEnergyUseWN(kBtu)","Electricity(kBtu)","GHGEmissions(MetricTonsCO2e)","GHGEmissionsIntensity(kgCO2e/ft2)"]]

Les colonnes sont renomées sans parenthèses pour le transfert vers la base de donnée SQL.

In [5]:
building_table.rename(columns={"SiteEnergyUseWN(kBtu)":"SiteEnergyUseWN_kBtu","Electricity(kBtu)":"Electricity_kBtu","GHGEmissions(MetricTonsCO2e)":"GHGEmissions_MetricTonsCO2e","GHGEmissionsIntensity(kgCO2e/ft2)":"GHGEmissionsIntensity_kgCO2eperft2"},inplace=True)

/tmp/ipykernel_5189/1374716298.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  building_table.rename(columns={"SiteEnergyUseWN(kBtu)":"SiteEnergyUseWN_kBtu","Electricity(kBtu)":"Electricity_kBtu","GHGEmissions(MetricTonsCO2e)":"GHGEmissions_MetricTonsCO2e","GHGEmissionsIntensity(kgCO2e/ft2)":"GHGEmissionsIntensity_kgCO2eperft2"},inplace=True)


# Création de la base de donnée SQL et de la table "Building".

On initialise la connection vers la base de donnée SQL "Seattle_Energy.db".

In [14]:
connection = sqlite3.connect("Seattle_Energy.db")
cursor = connection.cursor()

Exportation du dataset Building vers une table Building.

In [ ]:
building_table.to_sql("Building",con=connection,index=False)

In [12]:
connection.commit()

On effectue un affichage test du contenu de la table Building.

In [18]:
query = "select * from Building"
cursor.execute(query)

In [19]:
res = cursor.fetchall()
for i in res:
    print(i)

(1, 'MAYFLOWER PARK HOTEL', 'Hotel', 1927, 1, 12.0, 88434, 65.0, 7097539.0, 3686160.0, 249.43, 2.64)
(2, 'PARAMOUNT HOTEL', 'Hotel', 1996, 1, 11.0, 103566, 51.0, 8765788.0, 3905411.0, 263.51, 2.38)
(3, 'WESTIN HOTEL', 'Hotel', 1969, 1, 41.0, 961990, 18.0, 75506272.0, 49762435.0, 2061.48, 1.92)
(5, 'HOTEL MAX', 'Hotel', 1926, 1, 10.0, 61320, 1.0, 28363444.0, 2769023.0, 1936.34, 31.38)
(8, 'WARWICK SEATTLE HOTEL', 'Hotel', 1980, 1, 18.0, 119890, 67.0, 15078243.0, 6066245.0, 507.7, 4.02)
(9, 'WEST PRECINCT (SEATTLE POLICE)', 'Other', 1999, 1, 2.0, 97288, None, 13045258.0, 7271004.0, 304.62, 2.81)
(10, 'CAMLIN WORLDMARK HOTEL', 'Hotel', 1926, 1, 11.0, 83008, 25.0, 6477493.0, 2679698.0, 208.46, 2.37)
(11, 'PARAMOUNT THEATER', 'Other', 1926, 1, 8.0, 102761, None, 7380086.0, 4108004.0, 199.99, 1.77)
(12, 'COURTYARD BY MARRIOTT - ALASKA BLDG SEATTLE', 'Hotel', 1904, 1, 15.0, 163984, 46.0, 13589025.0, 7355649.0, 331.61, 1.83)
(13, 'LYON BUILDING', 'Mid-Rise Multifamily', 1910, 1, 6.0, 63712, 2.

(26267, 'CHARBERN APTS', 'Mid-Rise Multifamily', 1925, 1, 8.0, 57504, 96.0, 1925181.0, 1132575.0, 36.86, 0.56)
(26269, 'HUDSON HOUSE', 'Low-Rise Multifamily', 1906, 1, 3.0, 25536, 97.0, 2054779.0, 389337.0, 74.28, 2.84)
(26281, 'WINDSOR COURT', 'Low-Rise Multifamily', 1989, 1, 4.0, 30720, 90.0, 941468.0, 888598.0, 6.19, 0.08)
(26282, 'WINDWATCH TOWNHOUSES CONDOMINIUM', 'Low-Rise Multifamily', 1987, 1, 4.0, 44832, 95.0, 881190.0, 816542.0, 5.69, 0.05)
(26283, 'WINDY HILLS CONDOMINIUM', 'Low-Rise Multifamily', 1968, 1, 4.0, 78120, 99.0, 1509428.0, 1366933.0, 9.53, 0.05)
(26289, 'COLUMBIA LUTHERAN HOME', 'Senior Care Community', 1961, 1, 2.0, 55974, 56.0, 6327231.0, 2690671.0, 191.76, 3.22)
(26305, 'WOODLAND COURT APARTMENTS', 'Low-Rise Multifamily', 1916, 1, 3.0, 28256, 94.0, 681867.0, 530509.0, 5.44, 0.11)
(26313, 'SANTORINI AT THE PARK APTS + OFFICE', 'Low-Rise Multifamily', 2001, 1, 4.0, 39235, None, 632534.0, 632560.0, 4.41, 0.04)
(26319, 'WING LUKE ELEMENTARY SCHOOL (SPS-DISTRICT)',

# Nettoyage du dataset pour préparer les autres tables

On utilise une compréhension de liste pour créer des listes qui contiendront la description du type d'énergie consommé par chaque bâtiment.

In [25]:
s = ["Steam" if x!= 0 else "" for x in Seattle_energy_dataframe["SteamUse(kBtu)"]]

In [26]:
e = ["Electricity" if x!= 0 else "" for x in Seattle_energy_dataframe["Electricity(kWh)"]]

In [28]:
g = ["Natural Gas" if x!= 0 else "" for x in Seattle_energy_dataframe["NaturalGas(kBtu)"]]

In [29]:
o = ["Other Fuel" if x!= 0 else "" for x in Seattle_energy_dataframe["OtherFuelUse(kBtu)"]]

On rassemble les listes dans une liste puis on transpose le dataframe pour avoir les valeurs des listes en colonnes.

In [32]:
energyUse_dataframe = pd.DataFrame([s,e,g,o]).transpose()

Les 4 colonnes sont concaténées pour obtenir une colonne avec les types d'énergie utilisées par chaque bâtiment.

In [38]:
energyUse_dataframe_consolidated = energyUse_dataframe[0].str.cat(energyUse_dataframe[1],sep=", ").str.cat(energyUse_dataframe[2],sep=", ").str.cat(energyUse_dataframe[3],sep=", ")

On fait un peu de nettoyage dans la colonne pour enlever les virgules superflues.

In [62]:
energyUse_precursor_table = energyUse_dataframe_consolidated.str.replace(", , ",", ").str.replace(", $","").str.replace("^, ","")

/tmp/ipykernel_5189/693990920.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  energyUse_precursor_table = energyUse_dataframe_consolidated.str.replace(", , ",", ").str.replace(", $","").str.replace("^, ","")


In [60]:
energyUse_precursor_table

0       Steam, Electricity, Natural Gas
1              Electricity, Natural Gas
2       Steam, Electricity, Natural Gas
3       Steam, Electricity, Natural Gas
4              Electricity, Natural Gas
                     ...               
3335                        Electricity
3336           Electricity, Natural Gas
3337           Electricity, Natural Gas
3338                        Electricity
3339                        Electricity
Name: 0, Length: 3340, dtype: object

On envoie le dataframe energyUse dans la base de donnée pour servir de base au traitement par une requête SQL.

In [63]:
energyUse_precursor_table.to_sql("EnergyUse_Precursor",con=connection,index=False)

3340